<center><h1 style="color:#D4AF37"> ⚡⚡ AEMR Outage Data Analysis Using SQL ⚡⚡</h1>

<img src = "https://images.squarespace-cdn.com/content/v1/551972d8e4b0d571edc2e2c8/8f3abd33-0916-4415-9b24-593eea1d1e3c/Screen+Shot+2022-04-20+at+12.41.24+PM.png">



    


<h1 style="color:#D4AF37"> What's the Business Problem? 💰</h1>

The American Energy Market Regulator (AEMR) is responsible for looking after the
United States of America’s domestic energy network. The regulator’s responsibility is to
ensure that America’s energy network remains reliable with minimal disruptions, which
are known as outages. 

There are four key types of outages:

● Consequential

● Forced 

● Opportunistic 

● Planned 

Recently, the AEMR management team has been increasingly aware of a large number
of energy providers that submitted outages over the 2016 and 2017 calendar years. The
management team has expressed a desire to have the following two areas of concern
addressed.As an analyst within the data and reporting team, I have been asked to address these
two immediate areas of concern, and provide insights that may be of interest to the management team. 

<b> A) Energy Stability and Market Outages
    <p>
B) Energy Losses and Market Reliability </b>



In [325]:
import pandas as pd
import sqlite3
import prettytable

# Set the default display format for prettytable to 'DEFAULT' (i.e., a simple table format)
prettytable.DEFAULT = 'DEFAULT'

# 1. Load CSV into a DataFrame
df = pd.read_csv("aemr.csv")
df['Start_Time'] = pd.to_datetime(df['Start_Time'])
df['End_Time'] = pd.to_datetime(df['End_Time'])
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4658 entries, 0 to 4657
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Unnamed: 0               4655 non-null   float64       
 1   EventID                  4655 non-null   float64       
 2   Start_Time               4655 non-null   datetime64[ns]
 3   End_Time                 4655 non-null   datetime64[ns]
 4   Outage_Duration_In_Days  4655 non-null   float64       
 5   Year                     4655 non-null   float64       
 6   Month                    4655 non-null   float64       
 7   Facility_Code            4655 non-null   object        
 8   Participant_Code         4655 non-null   object        
 9   Status                   4655 non-null   object        
 10  Outage_Reason            4655 non-null   object        
 11  Energy_Lost_MW           4655 non-null   float64       
 12  Description_Of_Outage    4655 non-

In [326]:
# 2. Connect to SQLite database (creates it if it doesn't exist)
conn = sqlite3.connect("aemr.db")

# 3. Write the DataFrame to a SQL table
df.to_sql("aemr", conn, if_exists="replace", index=False)

# 4. (Optional) Query the table to check
#result = pd.read_sql("SELECT * FROM aemr LIMIT 5", conn)
#print(result)

# 5. Close the connection
conn.close()

<h3 style="color:#D4AF37"> ⚡ Part I. Energy Stability & Market Outages ⚡ </h3>

Energy stability is one of the key themes the AEMR management team cares about. To ensure energy security and reliability, AEMR needs to understand the following:
<p>
<b>

<li> What are the most common outage types and how long do they tend to last? 
<li> How frequently do the outages occur? 
<li> Are there any energy providers which have more outages than their peers which may be indicative of being unreliable? 
    
<p>

<u style="color:Green"> Please note that throughout the entire case study, we are interested ONLY in the Outages where Status = Approved. Please ignore other types of Outages such as cancelled or not approved.</u>

<h3 style="color:#D4AF37"> Question One </h3> 

<b> Write a SQL Statement to `COUNT` the number of valid (i.e. `Status = Approved`) Outage Events sorted by their respective `Outage_Reason` (i.e. `Forced`, `Consequential`, `Scheduled`, `Opportunistic`) over the 2016 & 2017 Periods. </b>

Do we notice anything regarding the trends for specific Outages over the 2016 / 2017 Period?

In [331]:
%%sql
select * from aemr limit 5;

 * sqlite:///AEMR.db
Done.


Unnamed: 0,EventID,Start_Time,End_Time,Outage_Duration_In_Days,Year,Month,Facility_Code,Participant_Code,Status,Outage_Reason,Energy_Lost_MW,Description_Of_Outage,Risk_Classification
0.0,1.0,2017-12-28 06:00:00,2017-12-28 10:00:00,0.17,2017.0,12.0,DNHR_DENMARK_WF1,DNHR,Approved,Consequential,1.44,a network outage in Denmark caused the windfarm to trip at 6:26 am,None
1.0,2.0,2017-12-26 09:00:00,2017-12-27 00:00:00,0.63,2017.0,12.0,COLLGAR_WF1,COLLGAR,Approved,Forced,30.0,Forced Outage - BS on Overhead line pole S-81 - CG10/11,High Risk
2.0,3.0,2017-12-31 09:00:00,2017-12-31 09:00:00,0.0,2017.0,12.0,AURICON_PNJ_U1,AURICON,Approved,Forced,5.703,Under generation - ambient conditions,High Risk
3.0,4.0,2017-12-31 08:30:00,2017-12-31 08:30:00,0.0,2017.0,12.0,AURICON_PNJ_U1,AURICON,Approved,Forced,5.085,Under generation - ambient conditions,High Risk
4.0,5.0,2017-12-31 08:00:00,2017-12-31 08:00:00,0.0,2017.0,12.0,AURICON_PNJ_U1,AURICON,Approved,Forced,5.02,Under generation - ambient conditions,High Risk


In [254]:
%%sql
SELECT COUNT(*) as Total_Number_Outages,
Outage_Reason,
Year
FROM aemr
WHERE Status = 'Approved'
GROUP BY Outage_Reason, Year
ORDER BY Year;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Outage_Reason,Year
181,Consequential,2016.0
1264,Forced,2016.0
106,Opportunistic Maintenance (Planned),2016.0
380,Scheduled (Planned),2016.0
127,Consequential,2017.0
1622,Forced,2017.0
102,Opportunistic Maintenance (Planned),2017.0
320,Scheduled (Planned),2017.0


<h4 style = "color:#5D3FD3">Findings: forced outage is the most common outage type and its total count has increased from 1264 to 1622 since year 2016.</h4>

<h3 style="color:#D4AF37"> Question Two </h3>



<b> i) Write a SQL Statement showing the `Total` of all Outage Types (Forced, Consequential, Scheduled, Opportunistic) where the `Status = Approved`, that occurred for both 2016 and 2017, grouped by `Year` and `Month`.  per month (i.e. 1 – 12). Order by `Year`, `Month`, `Total_Number_Outages` in Descending Order.


ii) Building on the query you write in i), group the results by `Outage Type`, `Year` and `Month`. This is so you can identify whether there is any outage type specifically increasing on a monthly basis when comparing 2016 to 2017. </b>


<b>⚠️Hint: You might find it helpful to create a small Common Table Expression to address these two questions! </b>

In [259]:
%%sql
SELECT Year, Month, Outage_Reason, COUNT(*)as Total_Number_Outages

FROM aemr

WHERE Status = 'Approved'
GROUP BY Month, Outage_Reason;

 * sqlite:///AEMR.db
Done.


Year,Month,Outage_Reason,Total_Number_Outages
2016.0,1.0,Consequential,36
2016.0,1.0,Forced,204
2017.0,1.0,Opportunistic Maintenance (Planned),18
2016.0,1.0,Scheduled (Planned),36
2017.0,2.0,Consequential,50
2016.0,2.0,Forced,246
2016.0,2.0,Opportunistic Maintenance (Planned),18
2016.0,2.0,Scheduled (Planned),63
2017.0,3.0,Consequential,26
2016.0,3.0,Forced,147


In [261]:
%%sql
SELECT Year, COUNT(*)as Total_Number_Outages

FROM aemr

WHERE Outage_Reason = 'Forced'
GROUP BY Year;

 * sqlite:///AEMR.db
Done.


Year,Total_Number_Outages
2016.0,1346
2017.0,1817


<h4 style = "color:#5D3FD3"> Findings: total number of outage events has rapidly increased from 2016 to 2017, especially among the forced outage type.</h4>


Hmm. Interesting. We've now identified a specific outage type is rapidly increasing from 2016 to 2017. However, frequency is just one aspect we care about. We also care about the <b> `duration` </b> of our ouatges.

In other words, if an outage is very short, we aren't as concerned. However, if the outage is very long, this then has the risk of threatening our energy supplies. We want to identify the problematic energy providers here. This leads us to our next question below. 

<h3 style="color:#D4AF37"> Question Three </h3>


<b>Write a SQL statement that calculates 1) The `Total_Number_Outage_Events` and 2) The <b> `Average Duration`</b> in <u>DAYS</u> for each `Participant Code` and `Outage Type` over the 2016 and 2017 Period where the `Status = Approved`. 
Order by `Total_Number_Outage_Events` in Descending Order, `Reason` and `Year`.  </b>
    


In [267]:
%%sql
SELECT Start_Time, End_Time, 
       JULIANDAY(End_Time) - JULIANDAY(Start_Time) AS Duration
FROM aemr
WHERE Outage_Reason = 'Forced'
  AND Start_Time IS NOT NULL
  AND End_Time IS NOT NULL
LIMIT 5;


 * sqlite:///AEMR.db
Done.


Start_Time,End_Time,Duration
2017-12-26 09:00:00,2017-12-27 00:00:00,0.625
2017-12-31 09:00:00,2017-12-31 09:00:00,0.0
2017-12-31 08:30:00,2017-12-31 08:30:00,0.0
2017-12-31 08:00:00,2017-12-31 08:00:00,0.0
2017-12-30 23:30:00,2017-12-30 23:30:00,0.0


In [269]:
%%sql
SELECT
Participant_Code, 
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) as Average_Outage_Duration_In_Days
#ROUND(AVG(ABS(JULIANDAY(datetime(End_Time)) - JULIANDAY(datetime(Start_Time)))), 4) as Average_Outage_Duration_In_Days
FROM aemr
WHERE Outage_Reason = 'Forced'
AND Start_Time IS NOT NULL
AND End_Time IS NOT NULL

GROUP BY Participant_Code
ORDER BY Average_Outage_Duration_In_Days DESC
LIMIT 5;



 * sqlite:///AEMR.db
(sqlite3.OperationalError) near "#ROUND(AVG(ABS(JULIANDAY(datetime(End_Time)": syntax error
[SQL: SELECT
Participant_Code, 
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) as Average_Outage_Duration_In_Days
#ROUND(AVG(ABS(JULIANDAY(datetime(End_Time)) - JULIANDAY(datetime(Start_Time)))), 4) as Average_Outage_Duration_In_Days
FROM aemr
WHERE Outage_Reason = 'Forced'
AND Start_Time IS NOT NULL
AND End_Time IS NOT NULL

GROUP BY Participant_Code
ORDER BY Average_Outage_Duration_In_Days DESC
LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)



We've identified participants who are having many outages, as well as participants who have been offline for the longest durations.

Armed with this information, it's important we're able to classify our participants accordingly based on reliability metrics of uptime.

We classify a participant based off the following criteria:
<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day)</li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours</li> 
</b>

<h3 style="color:#D4AF37"> Question Four </h3>

<b> Using the above criteria for context, write a SQL Statement that <u> classifies each participant code as either `High Risk`, `Medium Risk` or `Low Risk` in a column called `Risk_Classification`</u> that is based off their Average Outage Duration Time. Please note that this is for all valid (i.e. `Where status = approved`) outage types (Forced, Consequential, Scheduled, Opportunistic) for <u>all</u> participant codes from 2016 to 2017. Order the results using `Average Duration Time In Days` in descending order. 




In [274]:
%%sql
WITH CTE_risk AS(
SELECT
Participant_Code, 
Year, 
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) as Average_Outage_Duration_In_Days

FROM aemr
WHERE Outage_Reason = 'Forced'

GROUP BY Outage_Reason, Year, Participant_Code
ORDER BY Total_Number_Outage_Events DESC
)

SELECT *,
CASE
    WHEN Average_Outage_Duration_In_Days >= 1 THEN 'High Risk'
    WHEN Average_Outage_Duration_In_Days BETWEEN 0.5 AND 1 THEN 'Medium Risk'
    WHEN Average_Outage_Duration_In_Days <= 0.5 THEN 'Low Risk'
    ELSE 'N/A'
END as Risk_Classification
FROM CTE_risk;

 * sqlite:///AEMR.db
Done.


Participant_Code,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
AURICON,2017.0,616,0.06,Low Risk
GW,2016.0,323,0.37,Low Risk
GW,2017.0,238,1.02,High Risk
AURICON,2016.0,225,0.12,Low Risk
AUXC,2016.0,208,0.08,Low Risk
MELK,2017.0,180,2.25,High Risk
TRMOS,2017.0,179,0.4,Low Risk
MELK,2016.0,170,0.85,Medium Risk
PUG,2017.0,149,0.22,Low Risk
AUXC,2017.0,124,0.02,Low Risk


Now that we've classified our participants as either `High Risk`, `Medium Risk` or `Low Risk`, we want to dig a little deeper.

Does it make sense that `Consequential`, `Opportunistic` or `Planned` aren't considered regarding the Risk Category?

Perhaps we should refine our category accordingly by ensuring we focus our Risk Category on labelling only `Forced` Outages as being a Risk. After all, Forced Outages are the unplanned outages that risk the security of the electricity grid.

Let's add two additional criteria to our classification considering `Total Number of Outage Events` and `Outage Type`.

We've summarised these below:

<b>
<li> High Risk - On average, the participant is unavailable for > 24 Hours (1 Day) OR the Total Number of Outage Events > 20 </li>
<li> Medium Risk - On average, the participant is unavailable between 12 and 24 Hours OR the Total Number of Outage Events is Between 10 and 20 </li>
<li> Low Risk - On average, the participant is unavailable for less than 12 Hours OR the Total Number of Outage Events < 10 </li> 
<li> If Outage Type is not forced, then N/A
</b>


<h3 style="color:#D4AF37"> Question Five </h3>

<b> Write a SQL Statement that <u> classifies each participant code as either `High Risk`, `Medium Risk` or `Low Risk` in a column called `Risk_Classification`</u> using the new classification criteria. Order the results using `Average Duration Time In Days` in descending order. 



In [280]:
%%sql
WITH CTE_risk AS(
SELECT
Participant_Code, 
Outage_Reason, 
Year, 
COUNT(*) as Total_Number_Outage_Events,
ROUND(AVG((ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time)))),2) as Average_Outage_Duration_In_Days

FROM aemr
WHERE Status = 'Approved' 

GROUP BY Outage_Reason, Year, Participant_Code
ORDER BY Total_Number_Outage_Events DESC
)

SELECT *,
CASE
    WHEN Average_Outage_Duration_In_Days >= 1 OR Total_Number_Outage_Events > 20 THEN 'High Risk'
    WHEN Average_Outage_Duration_In_Days BETWEEN 0.5 AND 1 OR Total_Number_Outage_Events BETWEEN 10 AND 20 THEN 'Medium Risk'
    WHEN Average_Outage_Duration_In_Days <= 0.5 OR Total_Number_Outage_Events <=10 THEN 'Low Risk'
    ELSE 'N/A' END 
    as Risk_Classification
FROM CTE_risk
WHERE Outage_Reason = 'Forced'
ORDER BY Average_Outage_Duration_In_Days DESC
LIMIT 20;

 * sqlite:///AEMR.db
Done.


Participant_Code,Outage_Reason,Year,Total_Number_Outage_Events,Average_Outage_Duration_In_Days,Risk_Classification
EUCT,Forced,2016.0,11,5.81,High Risk
WGUTD,Forced,2017.0,8,3.44,High Risk
MELK,Forced,2017.0,177,2.28,High Risk
ENRG,Forced,2016.0,21,2.24,High Risk
COLLGAR,Forced,2017.0,45,1.38,High Risk
PJRH,Forced,2016.0,81,1.22,High Risk
KORL,Forced,2017.0,76,1.21,High Risk
COLLGAR,Forced,2016.0,29,1.11,High Risk
GW,Forced,2017.0,227,1.06,High Risk
PUG,Forced,2016.0,24,0.92,High Risk


<h4 style = "color:#5D3FD3"> Findings: the top three venders who has the longest average_duration_in_days are EUCT, WGUTD, and MELK.  </h4>

<h3 style="color:#D4AF37"> ⚡ Part II. Energy Losses & Market Reliability ⚡ </h3>

When an energy provider provides energy to the market, they are making a commitment to the market and saying; “We will supply X amount of energy to the market under a contractual obligation.” However, in a situation where the outages are Forced, the energy provider intended to provide energy but are unable to provide energy and are forced offline. <b style="color:Maroon">If many energy providers are forced offline at the same time it could cause an energy security risk that AEMR needs to mitigate. </b>

To ensure this doesn’t happen, the AEMR is interested in exploring the following questions:

<li> Of the outage types in 2016 and 2017, what percent were Forced Outage(s)?
<li> What was the average duration for a forced outage during both 2016 and 2017? Have we seen an increase in the average duration of forced outages?
<li> Which energy providers tended to have the largest number of forced outages?
    
<p>

<b> We'll examine this in the questions below. </b>
    
<img src = "https://media.istockphoto.com/id/1281821795/photo/market-stock-graph-and-information-with-city-light-and-electricity-and-energy-facility-banner.jpg?s=612x612&w=0&k=20&c=RSN5LqeMW28HW10aA190_DWR5YJ5tG2wixHFPBV3uZE=">

<h3 style="color:#D4AF37"> Question Six </h3>

<b> Write a SQL Statement to calculate the proportion of Forced Outages that have occurred over the 2016 - 2017 Period.
Do we observe any particular increases regarding any Outage Types over this period? </b>

In [287]:
%%sql
SELECT
Year,
COUNT(*) as Total_Number_Outages,
SUM(CASE 
    WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END) as Total_Number_Forced_Outage_Events,

ROUND(SUM(CASE WHEN Outage_Reason = 'Forced' THEN 1 ELSE 0 END)*1.0 / COUNT(*)*100, 2) as Pct_Outage_Forced

FROM aemr
WHERE Status = 'Approved'
GROUP BY Year;

 * sqlite:///AEMR.db
Done.


Year,Total_Number_Outages,Total_Number_Forced_Outage_Events,Pct_Outage_Forced
2016.0,1931,1264,65.46
2017.0,2171,1622,74.71


<h4 style = "color:#5D3FD3"> Findings: percentage of outage has gone up from 65.46% to 74.71% since 2016.</h4>


Great. It's clear to see now that `Forced Outages` are problematic for us. Not only are they the only outage type that generates financial losses as the Outage is unplanned, it seems there is a number of Energy Participants who have been having a significantly high number of Outages.

Now what can we do about this? 

Let's break our analysis down into Macro and Micro Analysis.
The total gives us the Overall Duration a participant is offline / has lost energy, however, it doesn't tell us how *frequently* this occurs. In other words, if we have one or two very big outages, it might contribute to very large totals.

However, perhaps an <b> average </b> can help us identify how big these Outages might really be, spread across the year!

Let's take a look.

<h3 style="color:#D4AF37"> Question Seven </h3>

Write a SQL Statement to calculate the `Total Number of Outages`, `Total Duration In Days`and `Total Energy Lost` of all valid `Outages` for each `participant code` and `facility_code`, sorted by `Total Energy Lost` in descending order and Ordered by the YEAR Category. 
    

In [293]:
%%sql
SELECT
COUNT(*) as Total_Number_Outages,
ROUND(SUM(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as Total_Duration_In_Days,
ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_Code,
Year
FROM aemr
WHERE Status = 'Approved'
GROUP BY Year, Facility_Code, Participant_Code, Outage_Reason
ORDER BY Total_Energy_Lost DESC, Year DESC;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
490,33.65,21639.55,Forced,AURICON,AURICON_PNJ_U1,2017.0
227,240.69,19326.56,Forced,GW,BW1_GREENWATERS_G2,2017.0
317,119.6,15751.38,Forced,GW,BW1_GREENWATERS_G2,2016.0
157,129.6,13771.07,Forced,MELK,MELK_G7,2016.0
208,15.06,10696.28,Forced,AURICON,AURICON_PNJ_U1,2016.0
177,404.15,10285.4,Forced,MELK,MELK_G7,2017.0
85,392.25,9668.79,Scheduled (Planned),MELK,MELK_G7,2016.0
69,34.06,9093.08,Forced,PMC,PMC_AG,2016.0
70,482.58,7499.28,Scheduled (Planned),MELK,MELK_G7,2017.0
46,87.02,6964.8,Scheduled (Planned),AURICON,AURICON_PNJ_U1,2016.0


In [295]:
%%sql
SELECT
COUNT(*) as Total_Number_Outages,
ROUND(SUM(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as Total_Duration_In_Days,
ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
Participant_Code,
Facility_Code,
Year
FROM aemr
WHERE Outage_Reason = 'Forced'
GROUP BY Facility_Code, Participant_Code,Year
ORDER BY Total_Energy_Lost DESC, Year DESC;

 * sqlite:///AEMR.db
Done.


Total_Number_Outages,Total_Duration_In_Days,Total_Energy_Lost,Participant_Code,Facility_Code,Year
616,37.5,30820.22,AURICON,AURICON_PNJ_U1,2017.0
238,242.98,20237.03,GW,BW1_GREENWATERS_G2,2017.0
323,120.46,16138.37,GW,BW1_GREENWATERS_G2,2016.0
170,143.83,15886.67,MELK,MELK_G7,2016.0
225,26.81,11949.19,AURICON,AURICON_PNJ_U1,2016.0
180,404.52,10903.1,MELK,MELK_G7,2017.0
72,34.46,9229.08,PMC,PMC_AG,2016.0
49,2.08,7284.86,PMC,PMC_AG,2017.0
81,98.79,5881.52,PJRH,PJRH_GT11,2016.0
179,72.44,5227.0,TRMOS,TIWEST_COG1,2017.0


<h4 style = "color:#5D3FD3"> Findings: the top 3 vendors who experienced massive energy losses are AURICON, MELK, and GW. </h4>


<h3 style="color:#D4AF37"> Question Eight </h3>

<b> Write a SQL Statement to calculate the `Average Duration In Days`and `Average Energy Lost` of all valid `FORCED OUTAGES` for each `participant code` and `facility_code` sorted by `Average Energy Lost` in descending order and Ordered by the YEAR Category. 


In [300]:
%%sql
SELECT
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as Avg_Duration_In_Days,
ROUND(AVG(Energy_Lost_MW),2) as Avg_Energy_Lost,
Outage_Reason,
Participant_Code,
Facility_Code,
Year

FROM aemr
WHERE Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Year
ORDER BY Avg_Duration_In_Days DESC
LIMIT 10;

 * sqlite:///AEMR.db
Done.


Avg_Duration_In_Days,Avg_Energy_Lost,Outage_Reason,Participant_Code,Facility_Code,Year
3.44,27.66,Forced,WGUTD,WEST_KALGOORLIE_GT2,2017.0
3.37,10.77,Forced,EUCT,GRASMERE_WF1,2016.0
2.25,60.57,Forced,MELK,MELK_G7,2017.0
1.95,46.33,Forced,ENRG,ENRG_KALGOORLIE_GT3,2016.0
1.34,61.79,Forced,KORL,KORL_GT3,2017.0
1.22,72.61,Forced,PJRH,PJRH_GT11,2016.0
1.21,65.29,Forced,COLLGAR,COLLGAR_WF1,2017.0
1.02,85.03,Forced,GW,BW1_GREENWATERS_G2,2017.0
0.98,129.72,Forced,COLLGAR,COLLGAR_WF1,2016.0
0.89,66.65,Forced,PJRH,PJRH_GT11,2017.0


<h4 style = "color:#5D3FD3"> Findings: the top three vendors who experienced the longest outage durations are WGUTD, EUCT, and MELK. However, it seems energy loss is not directly related to longer outage durations. </h4>


<h3 style="color:#D4AF37"> Question Nine </h3>

<b> Write a SQL Statement to calculate the `Average Energy Lost` and `Total Energy Lost` for each `Facility Code` and `Participant Code` across both the 2016 and 2017 periods when the `Outage_Reason` is set to Forced. Upon completion of this, calculate the <u> percentage </u> of energy lost due to forced outages for each `Facility_Code`. Please ORDER the query by `Total Energy Lost` from 2016 to 2017.
    
From your analysis, which participants have contributed the most to the Energy Lost due to Forced Outages?


In [305]:
%%sql
SELECT
ROUND(AVG(Energy_Lost_MW),2) as Avg_Energy_Lost,
ROUND(SUM(Energy_Lost_MW),2) as Total_Energy_Lost,
ROUND(SUM(Energy_Lost_MW) / (SELECT SUM(Energy_Lost_MW) FROM aemr WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2) as Pct_Energy_Loss,
Outage_Reason,
Participant_Code,
Facility_Code,
Year

FROM aemr
WHERE Outage_Reason = 'Forced' AND Status = 'Approved'
GROUP BY Participant_Code, Facility_Code, Year
ORDER BY Total_Energy_Lost DESC;


 * sqlite:///AEMR.db
Done.


Avg_Energy_Lost,Total_Energy_Lost,Pct_Energy_Loss,Outage_Reason,Participant_Code,Facility_Code,Year
44.16,21639.55,14.21,Forced,AURICON,AURICON_PNJ_U1,2017.0
85.14,19326.56,12.69,Forced,GW,BW1_GREENWATERS_G2,2017.0
49.69,15751.38,10.34,Forced,GW,BW1_GREENWATERS_G2,2016.0
87.71,13771.07,9.04,Forced,MELK,MELK_G7,2016.0
51.42,10696.28,7.02,Forced,AURICON,AURICON_PNJ_U1,2016.0
58.11,10285.4,6.75,Forced,MELK,MELK_G7,2017.0
131.78,9093.08,5.97,Forced,PMC,PMC_AG,2016.0
72.61,5881.52,3.86,Forced,PJRH,PJRH_GT11,2016.0
141.21,5648.44,3.71,Forced,PMC,PMC_AG,2017.0
29.17,5016.67,3.29,Forced,TRMOS,TIWEST_COG1,2017.0


<h3 style="color:#D4AF37"> Question Ten </h3>

<b>Having identified the top 3 participants by Total Energy Loss being `GW`, `MELK` and `Auricon`; Write a SQL Statement calculating the `Total_Energy_Lost` each of these three `Participant_Codes` and the `Facility_Code`. Additionally, identify the `Description_Of_Outage` associated with the highest `Total_Energy_Lost` for each of the `Participant_Codes` and `Facility_Code` for each of the three participants. 
    
<u>Lastly, calculate the percentage of Energy Loss, attributed to these reasons!</u> </b>



In [309]:
%%sql
SELECT *
FROM(
SELECT
Participant_Code,
Facility_Code,
Description_Of_Outage,
ROUND(AVG(ABS(JULIANDAY(End_Time) - JULIANDAY(Start_Time))),2) as AVG_Duration_In_Days,
ROUND(SUM(Energy_Lost_MW), 2) as Total_Energy_Lost,
ROUND(SUM(Energy_Lost_MW) / (SELECT SUM(Energy_Lost_MW) FROM aemr WHERE Status = 'Approved' AND Outage_Reason = 'Forced')*100,2) as Pct_Energy_Loss,
RANK() OVER (PARTITION BY Participant_Code, Facility_Code ORDER BY SUM(Energy_Lost_MW) DESC) as Rank_MW_Lost
FROM aemr
WHERE Participant_Code IN ('GW', 'MELK', 'AURICON') AND Status= 'Approved' AND Outage_Reason = 'Forced'
GROUP BY Participant_Code, Facility_Code, Description_Of_Outage
ORDER BY Participant_Code, Total_Energy_Lost DESC)

WHERE Rank_MW_Lost = 1

 * sqlite:///AEMR.db
Done.


Participant_Code,Facility_Code,Description_Of_Outage,AVG_Duration_In_Days,Total_Energy_Lost,Pct_Energy_Loss,Rank_MW_Lost
AURICON,AURICON_PNJ_U1,Full unit trip,0.0,6033.87,3.96,1
GW,BW1_GREENWATERS_G2,Operational Issues caused real time forced outage.,0.61,28687.54,18.84,1
MELK,MELK_G7,Safety Issues,11.65,1100.0,0.72,1


<h3 style = "color:#5D3FD3"> Findings: the 3 vendors with the longest average durations are AURICON, GW, and MELK.</h3>


That's a Wrap! I've now analyzed all the data I need to answer Management's Questions.
The next part of my analysis is to create my story in Tableau using the insights I've gathered from my SQL Analysis! 

Let's now switch to Part II of Analysis in Tableau!!
